In [2]:
import json
import tkinter as tk
from tqdm import tqdm
from IPython.display import clear_output
from database_connect import get_database

In [3]:
db = get_database()

In [4]:
def update_item_details(item, category, tags):
    query = {'item': str(item).lower()}
    insert = {'item': str(item).lower(), 'category': str(category).lower(), 'tags': [str(x).lower() for x in tags]}

    if db.item_details.find_one(query) is not None:
        result = db.item_details.replace_one(query, insert)
    else:
        result = db.item_details.insert_one(insert)
    
    return result

### New Entry Process

In [5]:
item_list = []
for recipe_details in tqdm(db.recipe_details.find()):
    for recipe_item in recipe_details['recipe']:
        item_list.append(recipe_item['item'])
item_set = list(set(item_list))

760it [00:04, 175.83it/s]


In [6]:
len(item_set)

467

In [15]:
def get_details(item):
    details = {}
    
    window = tk.Tk()
    window.title('My Window')
    window.geometry('100x250')

    l = tk.Label(window, bg='white', width=20, text=item)
    l.pack()
    
    def update(args):
        if category.get() == 'Meat':
            vegan.set(0)
            vegetarian.set(0)
            pescatarian.set(0)
        elif category.get() == 'Seafood':
            vegan.set(0)
            vegetarian.set(0)
        elif category.get() == 'Bakery':
            vegan.set(0)
            gluten_free.set(0)
        elif category.get() == 'Dairy':
            vegan.set(0)
        elif category.get() == 'Grains' or category.get() == 'Dry Goods':
            gluten_free.set(0)
    
    def submit():
        details['category'] = category.get()
        details['tags'] = []
        
        if vegan.get() == 1:
            details['tags'].append('vegan')
        if vegetarian.get() == 1:
            details['tags'].append('vegetarian')
        if pescatarian.get() == 1:
            details['tags'].append('pescatarian')
        if gluten_free.get() == 1:
            details['tags'].append('gluten free')
        window.destroy()

    categories = ["Produce", "Meat", "Seafood", "Bakery", "Dairy", "Frozen", "Grains", "Canned Goods", "Dry Goods", "Snacks", "Sauces", "Oils", "Spices", "Beverages", "Other"]
    category = tk.StringVar()
    vegan = tk.IntVar(value=1)
    vegetarian = tk.IntVar(value=1)
    pescatarian = tk.IntVar(value=1)
    gluten_free = tk.IntVar(value=1)
    o = tk.OptionMenu(window, category, *categories, command=update)
    o.pack()
    c1 = tk.Checkbutton(window, text='Vegan',variable=vegan, onvalue=1, offvalue=0)
    c1.pack()
    c2 = tk.Checkbutton(window, text='Vegetarian',variable=vegetarian, onvalue=1, offvalue=0)
    c2.pack()
    c3 = tk.Checkbutton(window, text='Pescatarian',variable=pescatarian, onvalue=1, offvalue=0)
    c3.pack()
    c4 = tk.Checkbutton(window, text='Gluten Free',variable=gluten_free, onvalue=1, offvalue=0)
    c4.pack()
    b1 = tk.Button(window, text ="Submit", command=submit)
    b1.pack()

    window.mainloop()
    
    return details

In [17]:
for i in range(len(item_set)):
    if len(list(db.item_details.find({'item': item_set[i]}))) == 0:
        clear_output()
        print(str(i+1) + '/' + str(len(item_set)))
        item_details = get_details(item_set[i])
        
        if item_details['category'] == '':
            for recipe_details in tqdm(db.recipe_details.find()):
                for recipe_item in recipe_details['recipe']:
                    if recipe_item['item'] == item_set[i]:
                        db.recipe_details.delete_one({'url': recipe_details['url']})
                        print('Deleted URL:\t' + recipe_details['url'])
                        break
        else:
            update_item_details(item_set[i], item_details['category'], item_details['tags'])

467/467
